# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism'

# upload all data to S3
data_s3 = sagemaker_session.upload_data(data_dir, bucket=bucket, key_prefix=prefix)

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

plagiarism/test.csv
plagiarism/train.csv
Test passed!


---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [6]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import BinaryClassifier
# pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [20]:

# your import and estimator code, here
from sagemaker.pytorch import PyTorch

output_path = 's3://{}/{}'.format(bucket, prefix)

estimator = PyTorch(entry_point='train.py',
                   source_dir='source_pytorch',
                   role=role,
                   framework_version='1.0',
                   train_instance_count=1,
                   train_instance_type='ml.p2.xlarge',
                   output_path=output_path,
                   sagemaker_session=sagemaker_session,
                   hyperparameters={
                       'input_features':5,
                       'hidden_dim':10,
                       'output_dim':1,
                       'epochs':10000
                   })


## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [21]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train':data_s3})

2019-07-16 07:51:24 Starting - Starting the training job...
2019-07-16 07:51:25 Starting - Launching requested ML instances......
2019-07-16 07:52:25 Starting - Preparing the instances for training.........
2019-07-16 07:54:21 Downloading - Downloading input data
2019-07-16 07:54:21 Training - Downloading the training image......
2019-07-16 07:55:12 Training - Training image download completed. Training in progress.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-07-16 07:55:13,069 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-07-16 07:55:13,099 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-07-16 07:55:16,115 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-07-16 07:55:16,422 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-07-16 07:55:16,422 sage

Epoch: 875, Loss: 0.3162241131067276
Epoch: 876, Loss: 0.3162109638963427
Epoch: 877, Loss: 0.31619779552732197
Epoch: 878, Loss: 0.31618466334683554
Epoch: 879, Loss: 0.3161715247801372
Epoch: 880, Loss: 0.3161583904709135
Epoch: 881, Loss: 0.3161452795778002
Epoch: 882, Loss: 0.3161321686846869
Epoch: 883, Loss: 0.31611907269273487
Epoch: 884, Loss: 0.3161059809582574
Epoch: 885, Loss: 0.3160929019962038
Epoch: 886, Loss: 0.3160798230341503
Epoch: 887, Loss: 0.3160667419433594
Epoch: 888, Loss: 0.31605367362499237
Epoch: 889, Loss: 0.31604062233652386
Epoch: 890, Loss: 0.31602759446416584
Epoch: 891, Loss: 0.3160145453044346
Epoch: 892, Loss: 0.31600151104586466
Epoch: 893, Loss: 0.31598848530224394
Epoch: 894, Loss: 0.315975478717259
Epoch: 895, Loss: 0.31596248916217257
Epoch: 896, Loss: 0.3159494868346623
Epoch: 897, Loss: 0.31593650792326244
Epoch: 898, Loss: 0.3159235119819641
Epoch: 899, Loss: 0.3159105394567762
Epoch: 900, Loss: 0.3158975839614868
Epoch: 901, Loss: 0.315884628

Epoch: 1813, Loss: 0.3094423881598881
Epoch: 1814, Loss: 0.30944076819079264
Epoch: 1815, Loss: 0.30952065118721556
Epoch: 1816, Loss: 0.3094345565353121
Epoch: 1817, Loss: 0.30943326226302553
Epoch: 1818, Loss: 0.30950418540409635
Epoch: 1819, Loss: 0.30942758704934803
Epoch: 1820, Loss: 0.3094265248094286
Epoch: 1821, Loss: 0.30948626356465475
Epoch: 1822, Loss: 0.3094212796006884
Epoch: 1823, Loss: 0.30942042597702574
Epoch: 1824, Loss: 0.30946715814726694
Epoch: 1825, Loss: 0.30941551285130636
Epoch: 1826, Loss: 0.30941481462546755
Epoch: 1827, Loss: 0.30944711182798657
Epoch: 1828, Loss: 0.30941019313676016
Epoch: 1829, Loss: 0.30940962050642284
Epoch: 1830, Loss: 0.30942629064832416
Epoch: 1831, Loss: 0.30940520550523487
Epoch: 1832, Loss: 0.3094047414405005
Epoch: 1833, Loss: 0.3094048351049423
Epoch: 1834, Loss: 0.30940050738198416
Epoch: 1835, Loss: 0.30940011356558117
Epoch: 1836, Loss: 0.3093828835657665
Epoch: 1837, Loss: 0.30939601361751556
Epoch: 1838, Loss: 0.30939570069

Epoch: 2749, Loss: 0.30790975264140535
Epoch: 2750, Loss: 0.3079594245978764
Epoch: 2751, Loss: 0.3079634883574077
Epoch: 2752, Loss: 0.3079449917588915
Epoch: 2753, Loss: 0.30791665613651276
Epoch: 2754, Loss: 0.30788485067231314
Epoch: 2755, Loss: 0.3079562463930675
Epoch: 2756, Loss: 0.3079056718519756
Epoch: 2757, Loss: 0.30795547153268543
Epoch: 2758, Loss: 0.30795965024403166
Epoch: 2759, Loss: 0.3079412430524826
Epoch: 2760, Loss: 0.3079129947083337
Epoch: 2761, Loss: 0.307881270136152
Epoch: 2762, Loss: 0.3079306994165693
Epoch: 2763, Loss: 0.3079022339412144
Epoch: 2764, Loss: 0.3079521209001541
Epoch: 2765, Loss: 0.307956371988569
Epoch: 2766, Loss: 0.3079380286591394
Epoch: 2767, Loss: 0.3079098186322621
Epoch: 2768, Loss: 0.30787815153598785
Epoch: 2769, Loss: 0.3079023063182831
Epoch: 2770, Loss: 0.3078992132629667
Epoch: 2771, Loss: 0.3079491662127631
Epoch: 2772, Loss: 0.3079534705196108
Epoch: 2773, Loss: 0.3079351484775543
Epoch: 2774, Loss: 0.30790697038173676
Epoch: 

Epoch: 3682, Loss: 0.3073543246303286
Epoch: 3683, Loss: 0.30743033971105305
Epoch: 3684, Loss: 0.30741279678685324
Epoch: 3685, Loss: 0.307466128042766
Epoch: 3686, Loss: 0.307484684245927
Epoch: 3687, Loss: 0.3074842372110912
Epoch: 3688, Loss: 0.3074745855161122
Epoch: 3689, Loss: 0.30746096798351835
Epoch: 3690, Loss: 0.30744597528662
Epoch: 3691, Loss: 0.3074308399643217
Epoch: 3692, Loss: 0.30741611974579947
Epoch: 3693, Loss: 0.30740202963352203
Epoch: 3694, Loss: 0.307388642004558
Epoch: 3695, Loss: 0.30737595898764475
Epoch: 3696, Loss: 0.30736395503793446
Epoch: 3697, Loss: 0.30735257480825695
Epoch: 3698, Loss: 0.3073538477931704
Epoch: 3699, Loss: 0.30741067443575176
Epoch: 3700, Loss: 0.3074636842523302
Epoch: 3701, Loss: 0.30748210847377777
Epoch: 3702, Loss: 0.3074816507952554
Epoch: 3703, Loss: 0.3074720416750227
Epoch: 3704, Loss: 0.30745848587581087
Epoch: 3705, Loss: 0.30744356981345583
Epoch: 3706, Loss: 0.30742851538317545
Epoch: 3707, Loss: 0.3074138696704592
Epoc

...

Epoch: 6491, Loss: 0.3071037083864212
Epoch: 6492, Loss: 0.30710042800222126
Epoch: 6493, Loss: 0.30709727108478546
Epoch: 6494, Loss: 0.30709423763411386
Epoch: 6495, Loss: 0.3070913276502064
Epoch: 6496, Loss: 0.30708853687558857
Epoch: 6497, Loss: 0.3070858546665737
Epoch: 6498, Loss: 0.30708327889442444
Epoch: 6499, Loss: 0.3070808010441916
Epoch: 6500, Loss: 0.30707841898713795
Epoch: 6501, Loss: 0.3070761242083141
Epoch: 6502, Loss: 0.30707391457898275
Epoch: 6503, Loss: 0.30707178158419474
Epoch: 6504, Loss: 0.3070697273526873
Epoch: 6505, Loss: 0.30706773911203655
Epoch: 6506, Loss: 0.3070658253771918
Epoch: 6507, Loss: 0.30706397124699186
Epoch: 6508, Loss: 0.30706217672143665
Epoch: 6509, Loss: 0.3070604460580008
Epoch: 6510, Loss: 0.30705876435552326
Epoch: 6511, Loss: 0.30705714225769043
Epoch: 6512, Loss: 0.3070555648633412
Epoch: 6513, Loss: 0.30705403642995016
Epoch: 6514, Loss: 0.30705255482878
Epoch: 6515, Loss: 0.30705111580235617
Epoch: 6516, Loss: 0.3070497193506786

Epoch: 7428, Loss: 0.3070083558559418
Epoch: 7429, Loss: 0.3070078747613089
Epoch: 7430, Loss: 0.3070074085678373
Epoch: 7431, Loss: 0.3070069466318403
Epoch: 7432, Loss: 0.30700649533952984
Epoch: 7433, Loss: 0.3070060504334314
Epoch: 7434, Loss: 0.3070056140422821
Epoch: 7435, Loss: 0.30700518616608213
Epoch: 7436, Loss: 0.30700476254735676
Epoch: 7437, Loss: 0.3070043474435806
Epoch: 7438, Loss: 0.30700394085475374
Epoch: 7439, Loss: 0.30700354065213886
Epoch: 7440, Loss: 0.3070031468357359
Epoch: 7441, Loss: 0.3070234443460192
Epoch: 7442, Loss: 0.3070297603096281
Epoch: 7443, Loss: 0.30706081645829336
Epoch: 7444, Loss: 0.3070795110293797
Epoch: 7445, Loss: 0.30708829845700947
Epoch: 7446, Loss: 0.3070911594799587
Epoch: 7447, Loss: 0.3070909253188542
Epoch: 7448, Loss: 0.30708920529910494
Epoch: 7449, Loss: 0.3070868211133139
Epoch: 7450, Loss: 0.30708418786525726
Epoch: 7451, Loss: 0.3070815013987677
Epoch: 7452, Loss: 0.3070788447346006
Epoch: 7453, Loss: 0.3070762668337141
Epo

Epoch: 8366, Loss: 0.30703097368989674
Epoch: 8367, Loss: 0.3070299114499773
Epoch: 8368, Loss: 0.30702887901238035
Epoch: 8369, Loss: 0.307027878505843
Epoch: 8370, Loss: 0.30702690354415346
Epoch: 8371, Loss: 0.3070259541273117
Epoch: 8372, Loss: 0.307025032384055
Epoch: 8373, Loss: 0.30702413405690876
Epoch: 8374, Loss: 0.30702326127461027
Epoch: 8375, Loss: 0.30702240765094757
Epoch: 8376, Loss: 0.30702157957213266
Epoch: 8377, Loss: 0.3070207727806909
Epoch: 8378, Loss: 0.3070199830191476
Epoch: 8379, Loss: 0.30701921454497744
Epoch: 8380, Loss: 0.3070184609719685
Epoch: 8381, Loss: 0.3070177286863327
Epoch: 8382, Loss: 0.3070170113018581
Epoch: 8383, Loss: 0.3070163152047566
Epoch: 8384, Loss: 0.3070156340088163
Epoch: 8385, Loss: 0.3070149655852999
Epoch: 8386, Loss: 0.3070143163204193
Epoch: 8387, Loss: 0.3070136776992253
Epoch: 8388, Loss: 0.3070130561079298
Epoch: 8389, Loss: 0.30701244516032083
Epoch: 8390, Loss: 0.3070118491138731
Epoch: 8391, Loss: 0.3070112700973238
Epoch

Epoch: 9305, Loss: 0.30699643918446134
Epoch: 9306, Loss: 0.306996117745127
Epoch: 9307, Loss: 0.30699580694947926
Epoch: 9308, Loss: 0.3069954982825688
Epoch: 9309, Loss: 0.30699519174439566
Epoch: 9310, Loss: 0.3069948958499091
Epoch: 9311, Loss: 0.30699460208415985
Epoch: 9312, Loss: 0.3069943125758852
Epoch: 9313, Loss: 0.30699402732508524
Epoch: 9314, Loss: 0.30699374633175985
Epoch: 9315, Loss: 0.30699347385338377
Epoch: 9316, Loss: 0.30699320137500763
Epoch: 9317, Loss: 0.3069929374115808
Epoch: 9318, Loss: 0.3069926713194166
Epoch: 9319, Loss: 0.30699241161346436
Epoch: 9320, Loss: 0.30699216042246136
Epoch: 9321, Loss: 0.30699190710272106
Epoch: 9322, Loss: 0.3069916580404554
Epoch: 9323, Loss: 0.306991417493139
Epoch: 9324, Loss: 0.30699117694582256
Epoch: 9325, Loss: 0.3069909406559808
Epoch: 9326, Loss: 0.30699071075235096
Epoch: 9327, Loss: 0.3069904787199838
Epoch: 9328, Loss: 0.30699025094509125
Epoch: 9329, Loss: 0.30699002742767334
Epoch: 9330, Loss: 0.3069898060389927


2019-07-16 07:57:19 Uploading - Uploading generated training model
2019-07-16 07:57:19 Completed - Training job completed
Billable seconds: 199
CPU times: user 2.02 s, sys: 190 ms, total: 2.21 s
Wall time: 6min 14s


## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [22]:
%%time

# uncomment, if needed
from sagemaker.pytorch import PyTorchModel


# deploy your model to create a predictor
model = PyTorchModel(model_data=estimator.model_data,
                        role=role,
                        framework_version='1.0',
                         entry_point='predict.py',
                         source_dir='source_pytorch')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')


--------------------------------------------------------------------------------------------------!CPU times: user 636 ms, sys: 25.8 ms, total: 662 ms
Wall time: 8min 16s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [33]:
# First: generate predicted, class labels
import numpy as np
test_y_preds = np.squeeze(predictor.predict(test_x)).astype('int')


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [34]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, test_y_preds)

print(accuracy)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

0.92

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

** Answer**:  
There are two false positives that the model produced and this might happen because the non-plagiarism answers use the same technical term to describe the problem, and it leads to being a match in the 1-gram which add the percentage that the answer is same as the source.


### Question 2: How did you decide on the type of model to use? 

** Answer**:  
I didn't choose the linear learner because I don't think that this is a linear problems. Thus I end up with neural net because it works for the non-linear problem such as the moon data set.



----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [35]:
# uncomment and fill in the line below!
predictor.delete_endpoint()

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [ ]:
# deleting bucket, uncomment lines below

# bucket_to_delete = boto3.resource('s3').Bucket(bucket)
# bucket_to_delete.objects.all().delete()

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!